# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
#!/usr/bin/python3
# top level script to run the show
import time
import PIL.Image

import image_classifier
from image_classifier import ImageClassifier

import gimbal_control as gc

## Image Classifier

In [ ]:
IC = ImageClassifier(graph_file='../Models/MobileNet_SSD_caffe/graph',
                     label_file='../Models/MobileNet_SSD_caffe/categories.txt')

In [ ]:
img_array = IC.take_picture_and_start_inference()
output_dict, bb, bb_scaled, inference_time_ms = IC.get_inference_result()

print(output_dict)
print(bb)
print(bb_scaled)
print(inference_time_ms)

In [ ]:
if bb == -1:
    img = PIL.Image.fromarray(img_array)
else:
    (x1, y1), (x2, y2) = bb
    img = PIL.Image.fromarray(img_array)
    img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))

img

## Gimbal Controller

In [ ]:
gc.turn_off_motors()

In [ ]:
gc.turn_on_motors()

In [ ]:
gc.send_angle_command(0, 0, 20)

In [ ]:
angles = gc.get_motor_angles()
print('Roll = %d, pitch = %d, yaw = %d' % (angles['roll'], angles['pitch'], angles['yaw']))

## A Basic Loop

In [ ]:
IC = ImageClassifier(graph_file='../Models/MobileNet_SSD_caffe/graph',
                     label_file='../Models/MobileNet_SSD_caffe/categories.txt')

In [ ]:
while True:
    print('.',end='')
    img_array = IC.take_picture_and_start_inference()
    output_dict, bb, bb_angles, inference_time_ms = IC.get_inference_result()

    if bb_angles == -1:
        img = PIL.Image.fromarray(img_array)
    else:
        # draw a box on the object
        (x1, y1), (x2, y2) = bb
        img = PIL.Image.fromarray(img_array)
        img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))
        img.show()

        # convert angles of object to centroids
        (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
        phi_yaw = (phi_x1 + phi_x2)/2
        phi_pitch = (phi_y1 + phi_y2)/2
        
        # get the current motor angles - need these to calculate offsets
        angles = gc.get_motor_angles()
        
        gc.send_angle_command(0, phi_pitch - angles['pitch'], phi_yaw - angles['yaw'])
        # old integrator approach
#         x_target += (x1+x2)/2
#         x_target = max(-20, min(20, x_target))
#         gc.send_angle_command(0, 0, x_target)
    time.sleep(0.05)

img